In [1]:
import numpy as np
import argparse
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
train_data = 'data/train'
test_data = 'data/test'

In [3]:
num_train = 28709
num_test = 7178
batch_size = 64
num_epoch = 50

In [4]:
train_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = train_data_gen.flow_from_directory(
    train_data, 
    target_size=(48,48),
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode='categorical')

Found 28709 images belonging to 7 classes.


In [6]:
test_generator = test_data_gen.flow_from_directory(
    test_data,
    target_size=(48,48),
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode='categorical')

Found 7178 images belonging to 7 classes.


In [7]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

In [8]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])

In [30]:
model_info = model.fit_generator(train_generator, steps_per_epoch=num_train // batch_size,
                                 epochs=num_epoch, validation_data=test_generator,
                                 validation_steps=num_test // batch_size)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 448 steps, validate for 112 steps
Epoch 1/50
448/448 [==============================] - 568s 1s/step - loss: 1.8042 - accuracy: 0.2564 - val_loss: 1.7295 - val_accuracy: 0.3270
Epoch 2/50
448/448 [==============================] - 264s 589ms/step - loss: 1.6371 - accuracy: 0.3639 - val_loss: 1.5477 - val_accuracy: 0.4116
Epoch 3/50
448/448 [==============================] - 243s 542ms/step - loss: 1.5332 - accuracy: 0.4082 - val_loss: 1.4631 - val_accuracy: 0.4404
Epoch 4/50
448/448 [==============================] - 223s 498ms/step - loss: 1.4607 - accuracy: 0.4389 - val_loss: 1.4017 - val_accuracy: 0.4632
Epoch 5/50
448/448 [==============================] - 245s 547ms/step - loss: 1.3990 - accuracy: 0.4669 - val_loss: 1.3493 - val_accuracy: 0.4847
Epoch 6/50
448/448 [==============================] - 218s 486ms/step - loss: 1.3490 - accuracy: 0.4890

In [31]:
model.save_weights('model.h5')

In [32]:
model.load_weights('model.h5')
# cv2.ocl.setUseOpenCL(False)
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

def rescale_frame(frame, percent=75):
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = rescale_frame(frame, 120)
    facecascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = facecascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    num_of_faces = len(faces)
    cnt = 0
    nxt = 30
    for (x, y, w, h) in faces:
        color = (255, 0, 0)  # BGR 0-255
        stroke = 2
        left_up_cord_x = x
        left_up_cord_y = y
        right_bottom_cord_x = x + w  # may say width of rectangle
        right_bottom_cord_y = y + h  # may say height of rectangle
        cv2.rectangle(frame, (left_up_cord_x, left_up_cord_y), (right_bottom_cord_x, right_bottom_cord_y), color, 2)

        roi_gray = gray[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
        prediction = model.predict(cropped_img)
        max_index = int(np.argmax(prediction))
        
        cv2.rectangle(frame, (left_up_cord_x-2, left_up_cord_y-30), (left_up_cord_x+25, left_up_cord_y), color, cv2.FILLED)
        cv2.putText(frame, str(cnt), (x+5, y-5), cv2.FONT_HERSHEY_SIMPLEX, .8, (255, 255, 255), 2, cv2.LINE_AA)
        em_text = "Face " + str(cnt) + ": " + emotion_dict[max_index]
        cv2.putText(frame, em_text, (10, 30+nxt), cv2.FONT_HERSHEY_SIMPLEX, .8, (0, 0, 255), 2, cv2.LINE_AA)
#         emotion_dict[max_index]
        cnt+=1
        nxt+=30
    text = "Total faces in frame: " + str(num_of_faces)
    cv2.putText(frame,text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, .8, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.imshow('Emotion Detection', frame)
    cnt = 0
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [10]:
(image_batch, label_batch) = test_generator.next()
loss, acc = model.evaluate(image_batch, label_batch, verbose=1)
print("accuracy: {:5.2f}%".format(100*acc))

64/64 [==============================] - 0s 7ms/sample - loss: 1.1408 - accuracy: 0.6250
accuracy: 62.50%
